# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [2]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [3]:
with open('data/litw-win.txt', 'r') as file:
    words = [s.split()[1] for s in file.read().split('\n')]

In [4]:
from nltk.metrics.distance import edit_distance

for s in text.split(' '):
    if s not in words:
        good_wrd = words[0]
        min_sz = edit_distance(s, good_wrd)
        for word in words[1:]:
            sz = edit_distance(s, word)
            if sz < min_sz:
                good_wrd = word
                min_sz = sz
        text = text.replace(s, good_wrd)
text

'с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий'

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [5]:
import re
from nltk.stem import SnowballStemmer
from razdel import sentenize
from razdel import tokenize

snb_stemmer_ru = SnowballStemmer('russian')
text = '''Считайте слова из файла litw-win.txt и запишите их в список words. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words.'''

tok = list(tokenize(text))
w = re.compile('^[а-яА-ЯёЁ]*')
' '.join([snb_stemmer_ru.stem(t.text) for t in tok if w.search(t.text)])

'счита слов из файл litw-win . txt и запиш их в список words . в зада предложен исправьт все опечатк , замен слов с опечатк на ближайш ( в смысл расстоян левенштейн ) к ним слов из списк words . счита , что в слов ест опечатк , есл дан слов не содерж в списк words .'

In [6]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()
pt = [morph.parse(t.text) for t in tok if w.search(t.text)] 
' '.join([w[0].normalized.word for w in pt])

'считать слово из файл litw-win . txt и записать они в список words . в задать предложение исправить всё опечатка , заменить слово с опечатка на близкий ( в смысл расстояние левенштейн ) к они слово из список words . считать , что в слово есть опечатка , если данный слово не содержаться в список words .'

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

In [7]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from collections import Counter

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [8]:
pd.read_csv('data/preprocessed_descriptions.csv').columns

Index(['name', 'preprocessed_descriptions'], dtype='object')

In [9]:
with open('data/unique_words.txt', 'r') as file:
    words = file.read().split(' ')

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [10]:
import random

for _ in range(5):
    pare = random.sample(words, 2)
    print(f'{pare[0]}, {pare[1]} = {edit_distance(*pare)}')

layerof, paul = 6
wanting, garland = 5
chili, or = 5
dimensions, tumblers = 8
buy, servng = 6


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [11]:
words

['amoretti',
 'thickening',
 'rose',
 'kielbases',
 'finsh',
 'befroe',
 'seaoned',
 'tilt',
 'ninths',
 'tempered',
 'fastest',
 'ripeness',
 'destem',
 'saucepancook',
 'beverage',
 'suace',
 'traingles',
 'minuets',
 'marbleized',
 'uoy',
 'arrabge',
 'reusing',
 'leche',
 'softness',
 'ultimate',
 'coating',
 'bratwursts',
 'template',
 'haze',
 'proof',
 'defatted',
 'tomatillo',
 'naam',
 'exposing',
 'proper',
 'estimated',
 'crem',
 'enclose',
 'spiciness',
 'wiches',
 'huey',
 'ovent',
 'lamingon',
 'male',
 'compressed',
 'cruet',
 'smoldering',
 'applied',
 'plums',
 'cooling',
 'hands',
 'stiring',
 'selected',
 'wrinkly',
 'adhering',
 'elements',
 'tenderly',
 'angostura',
 'catsup',
 'carnitas',
 'filberts',
 'balloon',
 'counting',
 'mineral',
 'lettuces',
 'bunched',
 'bore',
 'snifter',
 'association',
 'lengths',
 'pared',
 'koechin',
 'loins',
 'sum',
 'wet',
 'rosemary',
 'protected',
 'whisky',
 'worcesteshire',
 'pinto',
 'porchetta',
 'pilaf',
 'deseeded',
 'len

In [12]:
def get_unique_words():
    with open('data/unique_words.txt', 'r') as file:
        return file.read().split(' ')

def back_wrds(main_word, k):
    closest = {}
    words = get_unique_words()
    for word in words:
        if word == '':
            continue
        distance = edit_distance(main_word, word)
        if len(closest) < k:
            closest[word] = distance
        elif max(closest.values()) > distance:
            for key, value in closest.items():
                if value == max(closest.values()):
                    del_key = key
                    continue
            del closest[key]
            closest[word] = distance
    return closest

            
            
back_wrds('amaretti', 4)

{'amoretti': 1, 'thickening': 9, 'rose': 7, 'shaking': 7}

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [13]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Токенизация английского текста
text = ' '.join(words)
tok = word_tokenize(text)

# Создание стеммера для английского языка
snb_stemmer_en = SnowballStemmer('english')
stemmed = [snb_stemmer_en.stem(t) for t in tok]

# Пример нормализации слов с помощью библиотеки NLTK
normalized = [word.lower() for word in tok]


In [14]:
pd.DataFrame({'stemmed_word': stemmed, 'normalized_word': normalized}, index=words[:-1])

,stemmed_word,normalized_word
amoretti,amoretti,amoretti
thickening,thicken,thickening
rose,rose,rose
kielbases,kielbas,kielbases
finsh,finsh,finsh
...,...,...
done,done,done
feeds,feed,feeds
indgredients,indgredi,indgredients
hulls,hull,hulls


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [15]:
# Загрузка данных
recipes = pd.read_csv('data/preprocessed_descriptions.csv')
recipes.columns = ['name', 'preprocessed_descriptions']

# Загрузка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Функция для удаления стоп-слов из текста
def remove_stopwords(text, stop_words):
    words = re.findall(r'\b\w+\b', str(text).lower())
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words

# Создание списка всех слов до и после удаления стоп-слов
all_words_before = []
all_words_after = []

for description in recipes['preprocessed_descriptions']:
    words_before = re.findall(r'\b\w+\b', str(description).lower())
    words_after = remove_stopwords(description, stop_words)
    all_words_before.extend(words_before)
    all_words_after.extend(words_after)

# Подсчет общего количества слов и стоп-слов
total_words = len(all_words_before)
stop_words_count = total_words - len(all_words_after)

# Доля стоп-слов
stop_words_ratio = stop_words_count / total_words
print(f'Доля стоп-слов: {stop_words_ratio:.2%}')

# Подсчет частоты слов
counter_before = Counter(all_words_before)
counter_after = Counter(all_words_after)

# Топ-10 слов до удаления стоп-слов
top_10_before = counter_before.most_common(10)
print("Топ-10 слов до удаления стоп-слов:")
for word, freq in top_10_before:
    print(f'{word}: {freq}')

# Топ-10 слов после удаления стоп-слов
top_10_after = counter_after.most_common(10)
print("Топ-10 слов после удаления стоп-слов:")
for word, freq in top_10_after:
    print(f'{word}: {freq}')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Артём\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Доля стоп-слов: 47.08%
Топ-10 слов до удаления стоп-слов:
the: 40413
a: 35131
and: 30585
i: 27945
this: 27181
to: 23598
it: 23300
is: 20306
of: 18405
for: 16023
Топ-10 слов после удаления стоп-слов:
recipe: 15198
make: 6438
time: 5287
use: 4652
great: 4522
like: 4276
easy: 4263
one: 4018
good: 3887
made: 3874


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [85]:
recps = recipes.sample(5)
descriptions = recps.preprocessed_descriptions
titles = recps.name

tfidf_vectorizer = TfidfVectorizer()

# Преобразуем описания в числовые векторы
tfidf_vectors = tfidf_vectorizer.fit_transform(descriptions)

# Выводим результат
tfidf_vectors_array = tfidf_vectors.toarray()
print(tfidf_vectors_array)

[[0.         0.         0.         0.         0.         0.20414271
  0.         0.         0.         0.         0.         0.
  0.         0.3048218  0.         0.         0.         0.
  0.         0.         0.         0.24592831 0.         0.
  0.         0.         0.         0.         0.3048218  0.
  0.         0.         0.17173129 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.3048218  0.         0.         0.         0.
  0.         0.         0.3048218  0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.3048218  0.20414271
  0.         0.         0.         0.         0.         0.
  0.3048218  0.         0.         0.         0.3048218  0.20414271
  0.         0.         0.         0.20414271 0.3048218  0.
  0.         0. 

In [76]:
def get_distance(array):
    """
    Функция считает векторное расстояние между точками в любом количестве измерений для любого количества точек
    
    Input: Принимает матрицу из координат точек
    Output: Возвращает матрицу размера NxN (N - количество точек) с расстояниями
    """
    for i in range(1, len(array)):
        if len(array[0]) != len(array[i]):
            raise ValueError
    shape = (len(array), len(array[0]))
    result = [[0 for _ in range(shape[0])] for _ in range(shape[0])]
    for j in range(shape[0] - 1):
        for k in range(j + 1, shape[0]):
            cnt = 0
            for i in range(shape[1]):
                cnt += (array[j][i] - array[k][i])**2
            cnt = cnt**0.5
            result[j][k] = cnt
            result[k][j] = cnt
    return result
            
            
res = get_distance(tfidf_vectors_array)
for i in res:
    for j in i:
        print(f"{j:>20}", end='|')
    print()

                   0|  1.4142135623730954|  1.4142135623730947|   1.414213562373095|   1.414213562373095|
  1.4142135623730954|                   0|  1.2943304913343476|  1.3725568812139899|  1.4142135623730954|
  1.4142135623730947|  1.2943304913343476|                   0|   1.414213562373095|   1.339217064165524|
   1.414213562373095|  1.3725568812139899|   1.414213562373095|                   0|   1.414213562373095|
   1.414213562373095|  1.4142135623730954|   1.339217064165524|   1.414213562373095|                   0|


3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [88]:
from scipy.spatial.distance import cosine

tfidf_vectors = tfidf_vectorizer.fit_transform(descriptions).toarray()

# Вычисляем косинусное расстояние между каждой парой рецептов
distance_matrix = pd.DataFrame(
    [[cosine(tfidf_vectors[i], tfidf_vectors[j]) for j in range(len(titles))] for i in range(len(titles))],
    index=titles,
    columns=titles
)

# Выводим таблицу расстояний
distance_matrix

name,sourdough jack bread,extra creamy mashed potatoes,sunday chicken stew,flaky deli slices,quick chicken rice veggie soup
name,,,,,
sourdough jack bread,0.000000,1.000000,0.960397,0.849313,0.811505
extra creamy mashed potatoes,1.000000,0.000000,1.000000,0.922126,0.923902
sunday chicken stew,0.960397,1.000000,0.000000,0.981421,0.926459
flaky deli slices,0.849313,0.922126,0.981421,0.000000,0.805773
quick chicken rice veggie soup,0.811505,0.923902,0.926459,0.805773,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).